In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import csv
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")


D:\Application\Anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv("data.csv",low_memory = False)
df

,Date,Station,Source,Depth,YSI_Depth,YSI_Temp,YSI_Salinity,YSI_DO,YSI_Turbidity,YSI_Chl,...,CtoN,DOC,DIC,NO3/NO2,NH4,DIN,TDN,DON,PO4,SiO2
0,2000/1/6,0,IMS,2,4.000,10.93,0.10,9.36,-7777.0,-7777.0,...,6.90,441.8,-7777.000000,779.296051,57.007864,836.303915,1320.963257,485.0,-9999.00000,96.903763
1,2000/1/6,0,IMS,1,0.100,11.15,0.10,9.64,-7777.0,-7777.0,...,5.46,439.4,4.870467,776.528748,59.663273,836.192021,1314.189941,478.0,-9999.00000,93.920674
2,2000/1/6,10,IMS,2,3.000,10.59,0.10,9.50,-7777.0,-7777.0,...,8.01,-7777.0,-7777.000000,769.291901,66.964248,836.256149,-7777.000000,-2222.0,-9999.00000,98.343296
3,2000/1/6,10,IMS,1,0.100,10.97,0.10,9.74,-7777.0,-7777.0,...,7.40,-7777.0,5.074685,775.346496,65.562847,840.909343,-7777.000000,-2222.0,-9999.00000,95.093953
4,2000/1/6,20,IMS,2,2.500,10.54,0.10,9.75,-7777.0,-7777.0,...,7.11,-7777.0,-7777.000000,778.716033,75.779327,854.495360,1342.243408,488.0,4.62645,115.384312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,2019/7/9,140,IMS,2,5.080,28.28,24.87,0.71,2.5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9963,2019/7/9,160,IMS,1,0.300,28.69,16.41,6.93,1.2,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9964,2019/7/9,160,IMS,2,7.022,28.27,27.31,0.86,5.6,3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9965,2019/7/9,180,IMS,1,0.373,28.62,17.90,6.44,0.7,3.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#df = df.replace(np.nan,-1)
df = df.replace("#REF!",np.nan)
df = df.replace(" ",np.nan)
df = df.replace('#DIV/0!',np.nan)
df = df.replace('#VALUE!',np.nan)
for i in range(1,9):
    df = df.replace(i*(-1111),np.nan)
df = df.replace(-9999,0)
df

,Date,Station,Source,Depth,YSI_Depth,YSI_Temp,YSI_Salinity,YSI_DO,YSI_Turbidity,YSI_Chl,...,CtoN,DOC,DIC,NO3/NO2,NH4,DIN,TDN,DON,PO4,SiO2
0,2000/1/6,0,IMS,2,4.000,10.93,0.10,9.36,NaN,NaN,...,6.90,441.8,NaN,779.296051,57.007864,836.303915,1320.963257,485.0,0.00000,96.903763
1,2000/1/6,0,IMS,1,0.100,11.15,0.10,9.64,NaN,NaN,...,5.46,439.4,4.870467,776.528748,59.663273,836.192021,1314.189941,478.0,0.00000,93.920674
2,2000/1/6,10,IMS,2,3.000,10.59,0.10,9.50,NaN,NaN,...,8.01,NaN,NaN,769.291901,66.964248,836.256149,NaN,NaN,0.00000,98.343296
3,2000/1/6,10,IMS,1,0.100,10.97,0.10,9.74,NaN,NaN,...,7.40,NaN,5.074685,775.346496,65.562847,840.909343,NaN,NaN,0.00000,95.093953
4,2000/1/6,20,IMS,2,2.500,10.54,0.10,9.75,NaN,NaN,...,7.11,NaN,NaN,778.716033,75.779327,854.495360,1342.243408,488.0,4.62645,115.384312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,2019/7/9,140,IMS,2,5.080,28.28,24.87,0.71,2.5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9963,2019/7/9,160,IMS,1,0.300,28.69,16.41,6.93,1.2,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9964,2019/7/9,160,IMS,2,7.022,28.27,27.31,0.86,5.6,3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9965,2019/7/9,180,IMS,1,0.373,28.62,17.90,6.44,0.7,3.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data = df.iloc[:,[0,1,3,4,5,6,7,8,9,11,12,13,16,17,18,19,20,21,22]]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9967 entries, 0 to 9966
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9967 non-null   object 
 1   Station        9967 non-null   object 
 2   Depth          9967 non-null   object 
 3   YSI_Depth      9911 non-null   float64
 4   YSI_Temp       9911 non-null   float64
 5   YSI_Salinity   9917 non-null   float64
 6   YSI_DO         9896 non-null   float64
 7   YSI_Turbidity  9415 non-null   float64
 8   YSI_Chl        9493 non-null   float64
 9   POC            9859 non-null   float64
 10  PN             9801 non-null   float64
 11  CtoN           9799 non-null   float64
 12  NO3/NO2        9700 non-null   float64
 13  NH4            9700 non-null   float64
 14  DIN            9707 non-null   float64
 15  TDN            8972 non-null   float64
 16  DON            8956 non-null   float64
 17  PO4            9700 non-null   float64
 18  SiO2    

In [5]:
# 缺失值比例
data.isnull().sum()/len(data)

Date             0.000000
Station          0.000000
Depth            0.000000
YSI_Depth        0.005619
YSI_Temp         0.005619
YSI_Salinity     0.005017
YSI_DO           0.007124
YSI_Turbidity    0.055383
YSI_Chl          0.047557
POC              0.010836
PN               0.016655
CtoN             0.016856
NO3/NO2          0.026788
NH4              0.026788
DIN              0.026086
TDN              0.099829
DON              0.101435
PO4              0.026788
SiO2             0.026889
dtype: float64

## station-depth classification

In [6]:
data['Station'].value_counts()

Station
30                        901
70                        899
120                       895
50                        889
100                       888
60                        888
160                       887
140                       877
0                         858
20                        858
180                       813
95                        102
96                        100
165                        80
10                         24
Neuse River at Kinston      2
130                         1
Streets Ferry Bridge        1
Fisher Landing              1
Camp Seafarer               1
Union Point Neuse           1
Union Point Trent           1
Name: count, dtype: int64

In [7]:
data = data.drop(data[data['Station']=='Neuse River at Kinston'].index)
data = data.drop(data[data['Station']=='Streets Ferry Bridge'].index)
data = data.drop(data[data['Station']=='Fisher Landing'].index)
data = data.drop(data[data['Station']=='Camp Seafarer'].index)
data = data.drop(data[data['Station']=='Union Point Neuse'].index)
data = data.drop(data[data['Station']=='Union Point Trent'].index)
data = data.drop(data[data['Station']=='4'].index)
data = data.drop(data[data['Station']=='95'].index)
data = data.drop(data[data['Station']=='96'].index)
data = data.drop(data[data['Station']=='165'].index)
data = data.drop(data[data['Station']=='10'].index)
data = data.drop(data[data['Station']=='130'].index)

data['Station'].value_counts()

Station
30     901
70     899
120    895
50     889
60     888
100    888
160    887
140    877
0      858
20     858
180    813
Name: count, dtype: int64

In [8]:
data['Depth'].value_counts()

Depth
1        4811
2        4803
21D        19
11D        19
1.0 m       1
Name: count, dtype: int64

In [9]:
data['Depth'].replace(['21D', '11D'], ['2', '1'],inplace=True)
data.drop(index=data[data['Depth']=='1.0 m'].index[0],inplace=True)

data['Depth'].value_counts()

Depth
1    4830
2    4822
Name: count, dtype: int64

## Weather data processing

In [10]:
weather_data = pd.read_csv('daily_weather.csv',low_memory = False)
weather_data['Date']

0         1/1/1994
1         1/2/1994
2         1/3/1994
3         1/4/1994
4         1/5/1994
           ...    
9491    12/27/2019
9492    12/28/2019
9493    12/29/2019
9494    12/30/2019
9495    12/31/2019
Name: Date, Length: 9496, dtype: object

In [12]:
weather_data = weather_data.replace('#DIV/0!',np.nan)
weather_data = weather_data.replace("#REF!",np.nan)
weather_data.Date = pd.to_datetime(weather_data.Date,format='%m/%d/%Y')
weather_data.NW14 = weather_data.NW14.astype(float)
weather_data.SW14 = weather_data.SW14.astype(float)
weather_data.N14 = weather_data.N14.astype(float)
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9496 entries, 0 to 9495
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    9496 non-null   datetime64[ns]
 1   ATMP    9340 non-null   float64       
 2   NW14    9496 non-null   float64       
 3   SW14    9496 non-null   float64       
 4   N14     9496 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 371.1 KB


In [13]:
weather_data

,Date,ATMP,NW14,SW14,N14
0,1994-01-01,8.260870,0.0,0.000000,0.000000
1,1994-01-02,12.066667,0.0,0.000000,0.000000
2,1994-01-03,9.004167,0.0,0.000000,0.000000
3,1994-01-04,10.541667,0.0,0.000000,0.000000
4,1994-01-05,5.966667,0.0,0.000000,0.000000
...,...,...,...,...,...
9491,2019-12-27,15.545833,0.0,112.870758,966.397813
9492,2019-12-28,16.975000,0.0,39.987285,966.397813
9493,2019-12-29,17.875000,0.0,39.987285,966.397813
9494,2019-12-30,18.804167,0.0,0.000000,966.397813


### merge weather data

In [14]:
# left join
data.Date = pd.to_datetime(data.Date,format='%Y/%m/%d')
# data = pd.merge(data,weather_data,how='left',on='Date')
# data

## river data

In [15]:
import datetime as dt
discharge_data = pd.read_csv('RiverDischarge1974_2020_revised_.csv')
discharge_data.Date = pd.to_datetime(discharge_data['Date'], format='%m/%d/%Y')
# discharge_data['Date'] = discharge_data['Date'].apply(lambda x: dt.datetime.strftime(x,'%Y/%#m/%#d'))
discharge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7300 entries, 0 to 7299
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          7300 non-null   datetime64[ns]
 1   flow (cfs)    7300 non-null   float64       
 2   ACC_FLOW_7    7300 non-null   float64       
 3   ACC_FLOW_14   7300 non-null   float64       
 4   ACC_FLOW_30   7300 non-null   float64       
 5   ACC_FLOW_40   7300 non-null   float64       
 6   ACC_FLOW_50   7300 non-null   float64       
 7   ACC_FLOW_60   7300 non-null   float64       
 8   ACC_FLOW_70   7300 non-null   float64       
 9   ACC_FLOW_80   7300 non-null   float64       
 10  ACC_FLOW_90   7300 non-null   float64       
 11  ACC_FLOW_180  7300 non-null   float64       
 12  ACC_FLOW_365  7300 non-null   float64       
dtypes: datetime64[ns](1), float64(12)
memory usage: 741.5 KB


In [16]:
discharge_data = discharge_data.iloc[:,[0,7]]
# data = pd.merge(data,discharge_data,how='left',on='Date')

discharge_data = discharge_data.replace('#DIV/0!',np.nan)
discharge_data = discharge_data.replace("#REF!",np.nan)
discharge_data
# data.isnull().sum()/len(data)

,Date,ACC_FLOW_60
0,2000-01-01,273120.0
1,2000-01-02,263270.0
2,2000-01-03,254320.0
3,2000-01-04,246090.0
4,2000-01-05,238300.0
...,...,...
7295,2019-12-22,120532.0
7296,2019-12-23,126512.0
7297,2019-12-24,131822.0
7298,2019-12-25,135782.0


## subsetting

In [17]:
def subsetting(data):
    groups = data.groupby(data.Station)
    sta0 = groups.get_group("0")
    sta20 = groups.get_group("20")
    sta30 = groups.get_group("30")
    sta50 = groups.get_group("50")
    sta60 = groups.get_group("60")
    sta70 = groups.get_group("70")
    sta100 = groups.get_group("100")
    sta120 = groups.get_group("120")
    sta140 = groups.get_group("140")
    sta160 = groups.get_group("160")
    sta180 = groups.get_group("180")


    groupsDepth0 = sta0.groupby(sta0.Depth)
    groupsDepth20 = sta20.groupby(sta20.Depth)
    groupsDepth30 = sta30.groupby(sta30.Depth)
    groupsDepth50 = sta50.groupby(sta50.Depth)
    groupsDepth60 = sta60.groupby(sta60.Depth)
    groupsDepth70 = sta70.groupby(sta70.Depth)
    groupsDepth100 = sta100.groupby(sta100.Depth)
    groupsDepth120 = sta120.groupby(sta120.Depth)
    groupsDepth140 = sta140.groupby(sta140.Depth)
    groupsDepth160 = sta160.groupby(sta160.Depth)
    groupsDepth180 = sta180.groupby(sta180.Depth)

# each station comprises of depth 1 and 2，
# which makes 11 * 2 = 22 subsets
    sta0_1 = groupsDepth0.get_group('1')
    sta0_2 = groupsDepth0.get_group('2')
    sta20_1 = groupsDepth20.get_group('1')
    sta20_2 = groupsDepth20.get_group('2')
    sta30_1 = groupsDepth30.get_group('1')
    sta30_2 = groupsDepth30.get_group('2')
    sta50_1 = groupsDepth50.get_group('1')
    sta50_2 = groupsDepth50.get_group('2')
    sta60_1 = groupsDepth60.get_group('1')
    sta60_2 = groupsDepth60.get_group('2')
    sta70_1 = groupsDepth70.get_group('1')
    sta70_2 = groupsDepth70.get_group('2')
    sta100_1 = groupsDepth100.get_group('1')
    sta100_2 = groupsDepth100.get_group('2')
    sta120_1 = groupsDepth120.get_group('1')
    sta120_2 = groupsDepth120.get_group('2')
    sta140_1 = groupsDepth140.get_group('1')
    sta140_2 = groupsDepth140.get_group('2')
    sta160_1 = groupsDepth160.get_group('1')
    sta160_2 = groupsDepth160.get_group('2')
    sta180_1 = groupsDepth180.get_group('1')
    sta180_2 = groupsDepth180.get_group('2')
    station_depth_df = [sta0_1,sta0_2
,sta20_1
,sta20_2
,sta30_1 
,sta30_2
,sta50_1
,sta50_2
,sta60_1
,sta60_2
,sta70_1 
,sta70_2
,sta100_1
,sta100_2
,sta120_1
,sta120_2
,sta140_1
,sta140_2
,sta160_1
,sta160_2
,sta180_1
,sta180_2]
    return station_depth_df

In [18]:
station_depth_df = subsetting(data)
# station_depth_df_test = subsetting(data_test)
# station_depth_df_test[0].Date
station_depth_df[0].Date

1      2000-01-06
27     2000-02-01
55     2000-02-09
82     2000-02-29
111    2000-03-14
          ...    
9857   2019-05-07
9879   2019-05-21
9901   2019-06-04
9923   2019-06-17
9945   2019-07-09
Name: Date, Length: 429, dtype: datetime64[ns]

In [19]:
for dataframe in station_depth_df:
    print("Station is %s. Depth is %s."%(dataframe['Station'].iloc[0],dataframe['Depth'].iloc[0]))
    print(dataframe.isnull().sum()/len(dataframe))
    print('-------------------------------------')

Station is 0. Depth is 1.
Date             0.000000
Station          0.000000
Depth            0.000000
YSI_Depth        0.016317
YSI_Temp         0.016317
YSI_Salinity     0.016317
YSI_DO           0.016317
YSI_Turbidity    0.058275
YSI_Chl          0.051282
POC              0.016317
PN               0.025641
CtoN             0.027972
NO3/NO2          0.030303
NH4              0.030303
DIN              0.030303
TDN              0.034965
DON              0.037296
PO4              0.030303
SiO2             0.030303
dtype: float64
-------------------------------------
Station is 0. Depth is 2.
Date             0.000000
Station          0.000000
Depth            0.000000
YSI_Depth        0.016317
YSI_Temp         0.016317
YSI_Salinity     0.016317
YSI_DO           0.016317
YSI_Turbidity    0.058275
YSI_Chl          0.051282
POC              0.018648
PN               0.027972
CtoN             0.027972
NO3/NO2          0.030303
NH4              0.030303
DIN              0.030303
TDN        

# Feature Engineering

In [20]:
# display all the columns
pd.set_option('display.max_columns', None)

station_depth_df[0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 1 to 9945
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           429 non-null    datetime64[ns]
 1   Station        429 non-null    object        
 2   Depth          429 non-null    object        
 3   YSI_Depth      422 non-null    float64       
 4   YSI_Temp       422 non-null    float64       
 5   YSI_Salinity   422 non-null    float64       
 6   YSI_DO         422 non-null    float64       
 7   YSI_Turbidity  404 non-null    float64       
 8   YSI_Chl        407 non-null    float64       
 9   POC            422 non-null    float64       
 10  PN             418 non-null    float64       
 11  CtoN           417 non-null    float64       
 12  NO3/NO2        416 non-null    float64       
 13  NH4            416 non-null    float64       
 14  DIN            416 non-null    float64       
 15  TDN            414 non-null

In [21]:
def past_now_shape(temp):
    temp.Date = pd.to_datetime(temp['Date'])
    temp['water_temperature_past'] = temp['YSI_Temp'].shift(1)
    temp['turbidity_past'] = temp['YSI_Turbidity'].shift(1)
    temp['Chl_past'] = temp['YSI_Chl'].shift(1)
    temp['POC_past'] = temp['POC'].shift(1)
    temp['PN_past'] = temp['PN'].shift(1)
    temp['CtoN_past'] = temp['CtoN'].shift(1)
    temp['NO3/NO2_past'] = temp['NO3/NO2'].shift(1)
    temp['DIN_past'] = temp['DIN'].shift(1)
    temp['TDN_past'] = temp['TDN'].shift(1)
    temp['DON_past'] = temp['DON'].shift(1)
    temp['PO4_past'] = temp['PO4'].shift(1)
    temp['SiO2_past'] = temp['SiO2'].shift(1)
    temp['NH4_past'] = temp['NH4'].shift(1)

    temp['DO_past'] = temp['YSI_DO'].shift(1)
    temp['DO_now'] = temp['YSI_DO']
    temp = temp.iloc[:,[0,1,2,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]]
    return temp
past_now_shape(station_depth_df[0]).info()


<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 1 to 9945
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    429 non-null    datetime64[ns]
 1   Station                 429 non-null    object        
 2   Depth                   429 non-null    object        
 3   water_temperature_past  421 non-null    float64       
 4   turbidity_past          403 non-null    float64       
 5   Chl_past                406 non-null    float64       
 6   POC_past                422 non-null    float64       
 7   PN_past                 418 non-null    float64       
 8   CtoN_past               417 non-null    float64       
 9   NO3/NO2_past            416 non-null    float64       
 10  DIN_past                416 non-null    float64       
 11  TDN_past                414 non-null    float64       
 12  DON_past                413 non-null    float64       

In [22]:
for i,df in enumerate(station_depth_df):
    station_depth_df[i] = past_now_shape(df)

In [25]:
for dataframe in station_depth_df:
    print("Station is %s. Depth is %s."%(dataframe['Station'].iloc[0],dataframe['Depth'].iloc[0]))
    print(dataframe.isnull().sum()/len(dataframe))
    print('-------------------------------------')

Station is 0. Depth is 1.
Date                      0.000000
Station                   0.000000
Depth                     0.000000
water_temperature_past    0.018648
turbidity_past            0.060606
Chl_past                  0.053613
POC_past                  0.016317
PN_past                   0.025641
CtoN_past                 0.027972
NO3/NO2_past              0.030303
DIN_past                  0.030303
TDN_past                  0.034965
DON_past                  0.037296
PO4_past                  0.030303
SiO2_past                 0.030303
NH4_past                  0.030303
DO_past                   0.018648
DO_now                    0.016317
dtype: float64
-------------------------------------
Station is 0. Depth is 2.
Date                      0.000000
Station                   0.000000
Depth                     0.000000
water_temperature_past    0.018648
turbidity_past            0.060606
Chl_past                  0.053613
POC_past                  0.018648
PN_past             

In [26]:
temp = station_depth_df[0]
temp.reset_index(drop=True,inplace=True)
temp.drop([0,0],inplace=True)
temp

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now
1,2000-02-01,0,1,11.15,NaN,NaN,222.15,47.55,5.46,776.528748,836.192021,1314.189941,478.0,0.000000,93.920674,59.663273,9.64,12.26
2,2000-02-09,0,1,2.82,NaN,NaN,1103.60,129.70,9.95,786.167450,877.206200,1313.772949,437.0,18.446947,27.631210,91.038750,12.26,11.23
3,2000-02-29,0,1,6.01,NaN,NaN,916.60,130.55,8.19,740.614106,763.458615,1155.559937,392.0,12.596420,21.707505,22.844510,11.23,9.05
4,2000-03-14,0,1,13.41,NaN,NaN,863.60,132.90,7.58,364.309173,371.978408,714.588379,343.0,10.014454,28.641857,7.669234,9.05,7.70
5,2000-03-27,0,1,15.90,NaN,NaN,352.30,45.00,9.13,675.008453,771.558037,1175.882080,404.0,22.072080,55.083531,96.549583,7.70,8.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,2019-05-07,0,1,19.70,10.7,6.3,684.00,100.00,7.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.16,5.20
425,2019-05-21,0,1,23.18,11.0,6.6,899.00,120.00,8.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.20,6.66
426,2019-06-04,0,1,28.29,7.9,7.8,654.00,125.00,6.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.66,7.68
427,2019-06-17,0,1,27.96,9.0,14.5,1479.00,260.00,6.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.68,6.10


In [27]:
for i in range(len(station_depth_df)):
    station_depth_df[i].reset_index(drop=True,inplace=True)
    station_depth_df[i].drop([0,0],inplace=True)
station_depth_df[8].head(50)

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now
1,2000-02-01,60,1,10.47,NaN,NaN,1846.10,289.75,7.43,444.049515,458.622455,NaN,NaN,0.000000,98.049100,14.572940,11.01,12.31
2,2000-02-09,60,1,3.41,NaN,NaN,1344.25,150.90,10.39,741.400300,835.215050,NaN,NaN,23.432557,46.732546,93.814750,12.31,11.84
3,2000-02-29,60,1,6.43,NaN,NaN,848.30,116.85,8.49,823.133450,886.354800,NaN,NaN,13.449027,26.401982,63.221350,11.84,10.30
4,2000-03-14,60,1,14.58,NaN,NaN,1502.75,240.15,7.30,411.497894,418.955108,NaN,NaN,16.043371,26.962816,7.457213,10.30,9.95
5,2000-03-27,60,1,14.23,NaN,NaN,1961.45,330.40,6.92,268.921768,278.282798,NaN,NaN,4.678274,35.407339,9.361029,9.95,9.52
6,2000-04-10,60,1,15.54,NaN,NaN,874.70,152.85,6.70,526.454498,562.222780,NaN,NaN,20.733933,47.000759,35.768282,9.52,11.20
7,2000-04-24,60,1,16.79,30.6,4.8,3340.67,494.33,7.88,86.560000,99.540000,NaN,NaN,19.970000,58.187200,12.980000,11.20,11.01
8,2000-05-08,60,1,18.27,NaN,NaN,2145.85,366.70,6.84,489.030000,559.620000,NaN,NaN,32.010000,67.827200,70.590000,11.01,10.15
9,2000-05-23,60,1,23.07,8.3,28.0,2249.95,414.05,6.34,168.960000,179.430000,NaN,NaN,9.580000,55.415100,10.470000,10.15,7.64
10,2000-06-05,60,1,24.62,NaN,NaN,1842.95,286.80,7.49,0.000000,5.910000,NaN,NaN,9.770000,43.568550,5.910000,7.64,7.70


### interpolation within station and depth to fit in RNN

In [28]:
import datetime
def skippingMissingDate(df):
    date_col = 'Date'

    dates = df[date_col].unique()
    dates = [pd.to_datetime(d) for d in dates]

    first_date = df[date_col].iloc[0]
    second_date = df[date_col].iloc[1]

    miss_dates = []
    if (second_date - first_date) > datetime.timedelta(days=1):
        delta = second_date - first_date
        miss_dates.extend([first_date + datetime.timedelta(days=d) for d in range(1, delta.days)])
    for i in range(2, len(dates)-2):
        if (dates[i+2] - dates[i])>datetime.timedelta(days=1):
            delta = dates[i+2] -dates[i]
            miss_dates.extend([dates[i+1]+ datetime.timedelta(days=d) for d in range(1,delta.days)])

    miss_dates.insert(0,first_date)

    for m_date in miss_dates:
        df = pd.concat([df,pd.DataFrame({date_col:m_date},index=[0])])
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(by=date_col).reset_index(drop=True)

    df.drop_duplicates(subset=date_col,keep='first',inplace=True)
    df.interpolate(method='linear',limit_direction='forward',inplace=True)
    return df
skippingMissingDate(station_depth_df[1]).head(50)

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now
0,2000-02-01,0,2,10.930000,NaN,NaN,302.800000,51.200000,6.900000,779.296051,836.303915,1320.963257,485.000000,0.000000,96.903763,57.007864,9.360000,12.040000
2,2000-02-02,NaN,NaN,9.913750,NaN,NaN,477.887500,67.800000,7.382500,780.209138,841.887632,1319.210968,477.625000,2.169336,88.822373,61.678494,9.695000,11.915000
3,2000-02-03,NaN,NaN,8.897500,NaN,NaN,652.975000,84.400000,7.865000,781.122226,847.471349,1317.458679,470.250000,4.338671,80.740983,66.349123,10.030000,11.790000
4,2000-02-04,NaN,NaN,7.881250,NaN,NaN,828.062500,101.000000,8.347500,782.035313,853.055066,1315.706390,462.875000,6.508007,72.659593,71.019752,10.365000,11.665000
5,2000-02-05,NaN,NaN,6.865000,NaN,NaN,1003.150000,117.600000,8.830000,782.948400,858.638782,1313.954101,455.500000,8.677342,64.578203,75.690382,10.700000,11.540000
6,2000-02-06,NaN,NaN,5.848750,NaN,NaN,1178.237500,134.200000,9.312500,783.861488,864.222499,1312.201813,448.125000,10.846678,56.496813,80.361012,11.035000,11.415000
7,2000-02-07,NaN,NaN,4.832500,NaN,NaN,1353.325000,150.800000,9.795000,784.774575,869.806216,1310.449524,440.750000,13.016014,48.415423,85.031641,11.370000,11.290000
8,2000-02-08,NaN,NaN,3.816250,NaN,NaN,1528.412500,167.400000,10.277500,785.687663,875.389933,1308.697235,433.375000,15.185349,40.334033,89.702270,11.705000,11.165000
9,2000-02-09,0,2,2.800000,NaN,NaN,1703.500000,184.000000,10.760000,786.600750,880.973650,1306.944946,426.000000,17.354685,32.252643,94.372900,12.040000,11.040000
10,2000-02-29,0,2,6.010000,NaN,NaN,1093.150000,144.250000,8.900000,748.195282,769.134657,1162.595215,393.000000,11.517490,33.870781,20.939375,11.040000,8.950000


In [29]:
for i in range(len(station_depth_df)):
    station_depth_df[i] = skippingMissingDate(station_depth_df[i])
station_depth_df[0].head(10)

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now
0,2000-02-09,0,1,2.8200,NaN,NaN,1103.60,129.7000,9.950,786.167450,877.206200,1313.772949,437.00,18.446947,27.631210,91.038750,12.2600,11.230
2,2000-02-10,NaN,NaN,2.9795,NaN,NaN,1094.25,129.7425,9.862,783.889783,871.518821,1305.862298,434.75,18.154421,27.335025,87.629038,12.2085,11.121
3,2000-02-11,NaN,NaN,3.1390,NaN,NaN,1084.90,129.7850,9.774,781.612116,865.831442,1297.951648,432.50,17.861894,27.038839,84.219326,12.1570,11.012
4,2000-02-12,NaN,NaN,3.2985,NaN,NaN,1075.55,129.8275,9.686,779.334448,860.144062,1290.040997,430.25,17.569368,26.742654,80.809614,12.1055,10.903
5,2000-02-13,NaN,NaN,3.4580,NaN,NaN,1066.20,129.8700,9.598,777.056781,854.456683,1282.130347,428.00,17.276842,26.446469,77.399902,12.0540,10.794
6,2000-02-14,NaN,NaN,3.6175,NaN,NaN,1056.85,129.9125,9.510,774.779114,848.769304,1274.219696,425.75,16.984315,26.150284,73.990190,12.0025,10.685
7,2000-02-15,NaN,NaN,3.7770,NaN,NaN,1047.50,129.9550,9.422,772.501447,843.081925,1266.309045,423.50,16.691789,25.854098,70.580478,11.9510,10.576
8,2000-02-16,NaN,NaN,3.9365,NaN,NaN,1038.15,129.9975,9.334,770.223779,837.394545,1258.398395,421.25,16.399263,25.557913,67.170766,11.8995,10.467
9,2000-02-17,NaN,NaN,4.0960,NaN,NaN,1028.80,130.0400,9.246,767.946112,831.707166,1250.487744,419.00,16.106736,25.261728,63.761054,11.8480,10.358
10,2000-02-18,NaN,NaN,4.2555,NaN,NaN,1019.45,130.0825,9.158,765.668445,826.019787,1242.577094,416.75,15.814210,24.965543,60.351342,11.7965,10.249


In [30]:
for i in range(len(station_depth_df)):
    station_depth_df[i]['turbidity_past']=station_depth_df[i]['turbidity_past'].fillna(station_depth_df[i]['turbidity_past'].mean())
    station_depth_df[i]['Chl_past']=station_depth_df[i]['Chl_past'].fillna(station_depth_df[i]['Chl_past'].mean()) 

station_depth_df[0].info()


<class 'pandas.core.frame.DataFrame'>
Index: 7078 entries, 0 to 14100
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    7078 non-null   datetime64[ns]
 1   Station                 174 non-null    object        
 2   Depth                   174 non-null    object        
 3   water_temperature_past  7078 non-null   float64       
 4   turbidity_past          7078 non-null   float64       
 5   Chl_past                7078 non-null   float64       
 6   POC_past                7078 non-null   float64       
 7   PN_past                 7078 non-null   float64       
 8   CtoN_past               7078 non-null   float64       
 9   NO3/NO2_past            7078 non-null   float64       
 10  DIN_past                7078 non-null   float64       
 11  TDN_past                7078 non-null   float64       
 12  DON_past                7078 non-null   float64     

In [31]:
for dataframe in station_depth_df:
    print("Station is %s. Depth is %s."%(dataframe['Station'].iloc[0],dataframe['Depth'].iloc[0]))
    print(dataframe.isnull().sum()/len(dataframe))
    print('-------------------------------------')


Station is 0. Depth is 1.
Date                      0.000000
Station                   0.975417
Depth                     0.975417
water_temperature_past    0.000000
turbidity_past            0.000000
Chl_past                  0.000000
POC_past                  0.000000
PN_past                   0.000000
CtoN_past                 0.000000
NO3/NO2_past              0.000000
DIN_past                  0.000000
TDN_past                  0.000000
DON_past                  0.000000
PO4_past                  0.000000
SiO2_past                 0.000000
NH4_past                  0.000000
DO_past                   0.000000
DO_now                    0.000000
dtype: float64
-------------------------------------
Station is 0. Depth is 2.
Date                      0.000000
Station                   0.976409
Depth                     0.976409
water_temperature_past    0.000000
turbidity_past            0.000000
Chl_past                  0.000000
POC_past                  0.000000
PN_past             

In [32]:
station_depth_df[0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7078 entries, 0 to 14100
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    7078 non-null   datetime64[ns]
 1   Station                 174 non-null    object        
 2   Depth                   174 non-null    object        
 3   water_temperature_past  7078 non-null   float64       
 4   turbidity_past          7078 non-null   float64       
 5   Chl_past                7078 non-null   float64       
 6   POC_past                7078 non-null   float64       
 7   PN_past                 7078 non-null   float64       
 8   CtoN_past               7078 non-null   float64       
 9   NO3/NO2_past            7078 non-null   float64       
 10  DIN_past                7078 non-null   float64       
 11  TDN_past                7078 non-null   float64       
 12  DON_past                7078 non-null   float64     

In [33]:

for i in range(len(station_depth_df)):
    station_depth_df[i] = pd.merge(station_depth_df[i],discharge_data,how='left',on='Date')
    station_depth_df[i] = pd.merge(station_depth_df[i],weather_data,how='left',on='Date')
    
station_depth_df[0]

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now,ACC_FLOW_60,ATMP,NW14,SW14,N14
0,2000-02-09,0,1,2.8200,9.357993,5.666389,1103.60,129.7000,9.950,786.167450,877.206200,1313.772949,437.00,18.446947,27.631210,91.038750,12.2600,11.230,299450.0,8.454167,545.631915,96.753472,538.863879
1,2000-02-10,NaN,NaN,2.9795,9.357993,5.666389,1094.25,129.7425,9.862,783.889783,871.518821,1305.862298,434.75,18.154421,27.335025,87.629038,12.2085,11.121,307270.0,11.554167,369.245526,96.753472,725.825583
2,2000-02-11,NaN,NaN,3.1390,9.357993,5.666389,1084.90,129.7850,9.774,781.612116,865.831442,1297.951648,432.50,17.861894,27.038839,84.219326,12.1570,11.012,314900.0,12.312500,369.245526,216.680519,603.370565
3,2000-02-12,NaN,NaN,3.2985,9.357993,5.666389,1075.55,129.8275,9.686,779.334448,860.144062,1290.040997,430.25,17.569368,26.742654,80.809614,12.1055,10.903,322300.0,9.250000,369.245526,216.680519,610.043591
4,2000-02-13,NaN,NaN,3.4580,9.357993,5.666389,1066.20,129.8700,9.598,777.056781,854.456683,1282.130347,428.00,17.276842,26.446469,77.399902,12.0540,10.794,329500.0,6.379167,369.245526,216.680519,610.043591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7073,2019-07-17,NaN,NaN,23.1800,11.000000,6.600000,899.00,120.0000,8.780,633.000000,646.500000,980.000000,334.00,15.800000,53.700000,13.500000,5.2000,6.660,132539.0,28.929167,0.000000,814.845560,0.000000
7074,2019-07-18,NaN,NaN,23.1800,11.000000,6.600000,899.00,120.0000,8.780,633.000000,646.500000,980.000000,334.00,15.800000,53.700000,13.500000,5.2000,6.660,135329.0,28.954167,0.000000,968.638196,0.000000
7075,2019-07-19,NaN,NaN,23.1800,11.000000,6.600000,899.00,120.0000,8.780,633.000000,646.500000,980.000000,334.00,15.800000,53.700000,13.500000,5.2000,6.660,138129.0,28.666667,0.000000,1109.824412,0.000000
7076,2019-07-20,NaN,NaN,23.1800,11.000000,6.600000,899.00,120.0000,8.780,633.000000,646.500000,980.000000,334.00,15.800000,53.700000,13.500000,5.2000,6.660,140169.0,28.545833,0.000000,1152.460926,0.000000


In [34]:
# output(station_depth_df)

In [35]:
temp = station_depth_df[0]
temp = temp.resample('14D', on='Date').last()
temp = temp.reset_index()
# temp.drop('index', axis=1, inplace=True)

temp

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now,ACC_FLOW_60,ATMP,NW14,SW14,N14
0,2000-02-09,0,1,4.893500,9.357993,5.666389,982.050000,130.2525,8.806000,756.557776,803.270270,1210.934491,407.750000,14.644104,23.780802,46.712494,11.590500,9.813000,387800.0,8.200000,70.371734,445.377149,691.195688
1,2000-02-23,0,1,6.010000,9.357993,5.666389,916.600000,130.5500,8.190000,740.614106,763.458615,1155.559937,392.000000,12.596420,21.707505,22.844510,11.230000,9.050000,426100.0,12.404167,66.335412,222.399765,202.124895
2,2000-03-08,0,1,13.410000,9.357993,5.666389,863.600000,132.9000,7.580000,364.309173,371.978408,714.588379,343.000000,10.014454,28.641857,7.669234,9.050000,7.700000,463470.0,10.925000,69.285721,845.717978,382.242237
3,2000-03-22,0,1,17.105714,9.357993,5.666389,393.842857,52.8000,8.928857,675.018916,766.833690,1166.741385,399.657143,23.178128,64.519945,91.814774,7.511429,7.784571,424900.0,19.870833,0.000000,403.471684,475.788660
4,2000-04-05,None,None,19.215714,9.357993,5.666389,466.542857,66.4500,8.576857,675.037225,758.566083,1150.745169,392.057143,25.113712,81.033671,83.528857,7.181429,7.302571,331260.0,17.287500,224.097602,886.276609,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,2019-05-22,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,324186.0,23.454167,0.000000,615.689984,0.000000
504,2019-06-05,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,285366.0,26.504167,0.000000,634.468526,0.000000
505,2019-06-19,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,159816.0,26.987500,32.158197,569.058685,67.607237
506,2019-07-03,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,130089.0,28.712500,0.000000,812.679749,0.000000


In [36]:
for i in range(len(station_depth_df)):
    station_depth_df[i]= station_depth_df[i].resample('14D', on='Date').last()
    station_depth_df[i] = station_depth_df[i].reset_index()
    
station_depth_df[0]

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now,ACC_FLOW_60,ATMP,NW14,SW14,N14
0,2000-02-09,0,1,4.893500,9.357993,5.666389,982.050000,130.2525,8.806000,756.557776,803.270270,1210.934491,407.750000,14.644104,23.780802,46.712494,11.590500,9.813000,387800.0,8.200000,70.371734,445.377149,691.195688
1,2000-02-23,0,1,6.010000,9.357993,5.666389,916.600000,130.5500,8.190000,740.614106,763.458615,1155.559937,392.000000,12.596420,21.707505,22.844510,11.230000,9.050000,426100.0,12.404167,66.335412,222.399765,202.124895
2,2000-03-08,0,1,13.410000,9.357993,5.666389,863.600000,132.9000,7.580000,364.309173,371.978408,714.588379,343.000000,10.014454,28.641857,7.669234,9.050000,7.700000,463470.0,10.925000,69.285721,845.717978,382.242237
3,2000-03-22,0,1,17.105714,9.357993,5.666389,393.842857,52.8000,8.928857,675.018916,766.833690,1166.741385,399.657143,23.178128,64.519945,91.814774,7.511429,7.784571,424900.0,19.870833,0.000000,403.471684,475.788660
4,2000-04-05,None,None,19.215714,9.357993,5.666389,466.542857,66.4500,8.576857,675.037225,758.566083,1150.745169,392.057143,25.113712,81.033671,83.528857,7.181429,7.302571,331260.0,17.287500,224.097602,886.276609,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,2019-05-22,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,324186.0,23.454167,0.000000,615.689984,0.000000
504,2019-06-05,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,285366.0,26.504167,0.000000,634.468526,0.000000
505,2019-06-19,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,159816.0,26.987500,32.158197,569.058685,67.607237
506,2019-07-03,None,None,23.180000,11.000000,6.600000,899.000000,120.0000,8.780000,633.000000,646.500000,980.000000,334.000000,15.800000,53.700000,13.500000,5.200000,6.660000,130089.0,28.712500,0.000000,812.679749,0.000000


In [43]:
for j in range(15):
    for i in range(1,len(station_depth_df)):
        station_depth_df[i]['ACC_FLOW_60']=station_depth_df[i]['ACC_FLOW_60'].astype('float') 
        station_depth_df[i].interpolate(method='pad',limit_direction='forward',inplace=True)
station_depth_df[0].isnull().sum()/len(station_depth_df[0])


Date                      0.000000
Station                   0.661417
Depth                     0.661417
water_temperature_past    0.000000
turbidity_past            0.000000
Chl_past                  0.000000
POC_past                  0.000000
PN_past                   0.000000
CtoN_past                 0.000000
NO3/NO2_past              0.000000
DIN_past                  0.000000
TDN_past                  0.000000
DON_past                  0.000000
PO4_past                  0.000000
SiO2_past                 0.000000
NH4_past                  0.000000
DO_past                   0.000000
DO_now                    0.000000
ACC_FLOW_60               0.000000
ATMP                      0.003937
NW14                      0.000000
SW14                      0.000000
N14                       0.000000
dtype: float64

In [44]:
station_depth_df[21]

,Date,Station,Depth,water_temperature_past,turbidity_past,Chl_past,POC_past,PN_past,CtoN_past,NO3/NO2_past,DIN_past,TDN_past,DON_past,PO4_past,SiO2_past,NH4_past,DO_past,DO_now,ACC_FLOW_60,ATMP,NW14,SW14,N14
0,2000-07-05,180,2,26.680000,3.749642,8.758759,1147.700000,200.900000,6.660000,2.16,31.350000,NaN,NaN,44.540000,66.795304,29.190000,5.950000,6.550000,74392.0,26.529167,0.000000,530.543116,24.969884
1,2000-07-19,180,2,26.870000,3.749642,8.758759,988.100000,184.350000,6.250000,0.00,22.230000,NaN,NaN,32.150000,49.490631,22.230000,6.550000,1.110000,89053.0,26.612500,0.000000,248.861734,0.000000
2,2000-08-02,180,2,26.685417,3.749642,8.758759,1186.577778,238.631944,5.802361,0.00,25.499579,NaN,NaN,66.259879,77.059290,25.499579,1.204861,6.111389,160517.0,26.012500,225.183497,863.998589,95.812759
3,2000-08-16,180,2,24.941250,3.749642,8.758759,1201.666667,240.479167,5.835417,0.00,22.833692,NaN,NaN,56.038181,71.431954,22.833692,2.532917,6.130833,192313.0,25.225000,0.000000,283.803698,58.505520
4,2000-08-30,180,2,23.197083,3.749642,8.758759,1216.755556,242.326389,5.868472,0.00,20.167804,NaN,NaN,45.816482,65.804619,20.167804,3.860972,6.150278,231435.0,25.304167,0.000000,101.075003,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2019-05-15,180,2,20.510000,1.900000,5.000000,1344.000000,230.000000,6.820000,4.72,18.520000,337.0,318.0,0.000000,51.500000,13.800000,1.420000,3.580000,354836.0,25.725000,0.000000,832.174588,102.800874
493,2019-05-29,180,2,20.510000,1.900000,5.000000,1344.000000,230.000000,6.820000,4.72,18.520000,337.0,318.0,0.000000,51.500000,13.800000,1.420000,3.580000,302786.0,24.716667,0.000000,546.274413,0.000000
494,2019-06-12,180,2,20.510000,1.900000,5.000000,1344.000000,230.000000,6.820000,4.72,18.520000,337.0,318.0,0.000000,51.500000,13.800000,1.420000,3.580000,214426.0,27.145833,0.000000,882.205043,56.699062
495,2019-06-26,180,2,20.510000,1.900000,5.000000,1344.000000,230.000000,6.820000,4.72,18.520000,337.0,318.0,0.000000,51.500000,13.800000,1.420000,3.580000,130768.0,26.095833,32.158197,626.252140,10.908175


In [50]:
concatenated_df = pd.concat(station_depth_df, ignore_index=True)


round the number

In [51]:
for column in concatenated_df.columns:
    if concatenated_df[column].dtype == 'float64':  
        concatenated_df[column] = concatenated_df[column].round(3)

concatenated_df = concatenated_df.applymap(lambda x: round(x, 3) if isinstance(x, float) else x)


filling the missing depth and station

In [52]:
missing_depth = concatenated_df['Depth'].isnull()
missing_station = concatenated_df['Station'].isnull()
concatenated_df.loc[missing_depth, 'Depth'] = 1
concatenated_df.loc[missing_station, 'Station'] = 0

In [53]:
concatenated_df.to_csv('final_interpolation_.csv', index=False)